Load in our labeled data

In [609]:
import pandas as pd
from dateutil import parser

dataset = 'data/output/vader/vader_bitcoin.xlsx'
output_dataset_name = 'ML_Data/BTC_vader.csv'
price_data = 'data/price/BTC.xlsx'
dates = 'data/dates.csv'

df = pd.read_excel(dataset)
df.columns = ['date', 'comment', 'label', 'compound_score']
df['date'] = pd.to_datetime(df['date'], dayfirst=True)

In [610]:
del df['comment']
del df['compound_score']

In [611]:
df.head()

,date,label
0,2019-05-04,0
1,2019-05-04,1
2,2019-05-04,0
3,2019-05-04,0
4,2019-05-04,1


Loop through all comments and create a new DataFrame that hosts the number of positives and negatives for each day.

In [612]:
rows = []
prices = []
grouped = df.groupby('date')
for date, group in grouped:
    positives = len(group[group['label'] == 1])
    negatives = len(group[group['label'] == 0])
        
    new_row = {
        'date': date,
        'positives': positives,
        'negatives': negatives,
        'ratio': positives / (positives + negatives)
    }
    
    rows.append(new_row)

In [613]:
new_df = pd.DataFrame(rows)

In [614]:
new_df.tail()

,date,negatives,positives,ratio
457,2019-04-30,18,8,0.307692
458,2019-05-01,18,11,0.379310
459,2019-05-02,39,21,0.350000
460,2019-05-03,43,20,0.317460
461,2019-05-04,21,11,0.343750


# Join with dates, so we make sure we have all dates

In [615]:
df_dates = pd.read_csv(dates)
df_dates['date'] = pd.to_datetime(df_dates['date'])
df_dates.tail(10)

,date
478,2018-01-10
479,2018-01-09
480,2018-01-08
481,2018-01-07
482,2018-01-06
483,2018-01-05
484,2018-01-04
485,2018-01-03
486,2018-01-02
487,2018-01-01


In [616]:
new_df = pd.merge(df_dates, new_df, on='date', how='outer')

In [617]:
new_df.fillna(0, inplace=True)

In [618]:
new_df.head(10)

,date,negatives,positives,ratio
0,2019-05-03,43.0,20.0,0.317460
1,2019-05-02,39.0,21.0,0.350000
2,2019-05-01,18.0,11.0,0.379310
3,2019-04-30,18.0,8.0,0.307692
4,2019-04-29,15.0,7.0,0.318182
5,2019-04-28,30.0,9.0,0.230769
6,2019-04-27,25.0,17.0,0.404762
7,2019-04-26,32.0,23.0,0.418182
8,2019-04-25,29.0,27.0,0.482143
9,2019-04-24,24.0,23.0,0.489362


Loop through new DataFrame and add 2 lags to each day. First we reverse order of DataFrame.

In [619]:
new_df = new_df.iloc[::-1]

lags = range(1, 8)

In [620]:
new_df = new_df.assign(**{
    '{} (t-{})'.format(col, t): new_df[col].shift(t)
    for t in lags
    for col in new_df
})

In [621]:
del new_df['date (t-1)']
del new_df['date (t-2)']
del new_df['date (t-3)']
del new_df['date (t-4)']
del new_df['date (t-5)']
del new_df['date (t-6)']
del new_df['date (t-7)']

In [622]:
new_df.fillna(0, inplace=True)
new_df.head(10)

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,ratio (t-4),negatives (t-5),positives (t-5),ratio (t-5),negatives (t-6),positives (t-6),ratio (t-6),negatives (t-7),positives (t-7),ratio (t-7)
488,2019-05-04,21.0,11.0,0.343750,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
487,2018-01-01,69.0,38.0,0.355140,21.0,11.0,0.343750,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
486,2018-01-02,75.0,37.0,0.330357,69.0,38.0,0.355140,21.0,11.0,0.343750,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
485,2018-01-03,24.0,25.0,0.510204,75.0,37.0,0.330357,69.0,38.0,0.355140,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
484,2018-01-04,25.0,14.0,0.358974,24.0,25.0,0.510204,75.0,37.0,0.330357,...,0.343750,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
483,2018-01-05,52.0,36.0,0.409091,25.0,14.0,0.358974,24.0,25.0,0.510204,...,0.355140,21.0,11.0,0.343750,0.0,0.0,0.000000,0.0,0.0,0.000000
482,2018-01-06,44.0,19.0,0.301587,52.0,36.0,0.409091,25.0,14.0,0.358974,...,0.330357,69.0,38.0,0.355140,21.0,11.0,0.343750,0.0,0.0,0.000000
481,2018-01-07,15.0,17.0,0.531250,44.0,19.0,0.301587,52.0,36.0,0.409091,...,0.510204,75.0,37.0,0.330357,69.0,38.0,0.355140,21.0,11.0,0.343750
480,2018-01-08,73.0,45.0,0.381356,15.0,17.0,0.531250,44.0,19.0,0.301587,...,0.358974,24.0,25.0,0.510204,75.0,37.0,0.330357,69.0,38.0,0.355140
479,2018-01-09,17.0,14.0,0.451613,73.0,45.0,0.381356,15.0,17.0,0.531250,...,0.409091,25.0,14.0,0.358974,24.0,25.0,0.510204,75.0,37.0,0.330357


In [623]:
new_df = new_df.sort_values(['date'],ascending=[False])

In [624]:
new_df.head(10)

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,ratio (t-4),negatives (t-5),positives (t-5),ratio (t-5),negatives (t-6),positives (t-6),ratio (t-6),negatives (t-7),positives (t-7),ratio (t-7)
488,2019-05-04,21.0,11.0,0.343750,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
0,2019-05-03,43.0,20.0,0.317460,39.0,21.0,0.350000,18.0,11.0,0.379310,...,0.318182,30.0,9.0,0.230769,25.0,17.0,0.404762,32.0,23.0,0.418182
1,2019-05-02,39.0,21.0,0.350000,18.0,11.0,0.379310,18.0,8.0,0.307692,...,0.230769,25.0,17.0,0.404762,32.0,23.0,0.418182,29.0,27.0,0.482143
2,2019-05-01,18.0,11.0,0.379310,18.0,8.0,0.307692,15.0,7.0,0.318182,...,0.404762,32.0,23.0,0.418182,29.0,27.0,0.482143,24.0,23.0,0.489362
3,2019-04-30,18.0,8.0,0.307692,15.0,7.0,0.318182,30.0,9.0,0.230769,...,0.418182,29.0,27.0,0.482143,24.0,23.0,0.489362,23.0,22.0,0.488889
4,2019-04-29,15.0,7.0,0.318182,30.0,9.0,0.230769,25.0,17.0,0.404762,...,0.482143,24.0,23.0,0.489362,23.0,22.0,0.488889,25.0,17.0,0.404762
5,2019-04-28,30.0,9.0,0.230769,25.0,17.0,0.404762,32.0,23.0,0.418182,...,0.489362,23.0,22.0,0.488889,25.0,17.0,0.404762,16.0,10.0,0.384615
6,2019-04-27,25.0,17.0,0.404762,32.0,23.0,0.418182,29.0,27.0,0.482143,...,0.488889,25.0,17.0,0.404762,16.0,10.0,0.384615,19.0,7.0,0.269231
7,2019-04-26,32.0,23.0,0.418182,29.0,27.0,0.482143,24.0,23.0,0.489362,...,0.404762,16.0,10.0,0.384615,19.0,7.0,0.269231,9.0,12.0,0.571429
8,2019-04-25,29.0,27.0,0.482143,24.0,23.0,0.489362,23.0,22.0,0.488889,...,0.384615,19.0,7.0,0.269231,9.0,12.0,0.571429,27.0,11.0,0.289474


# Merge with price data

In [625]:
df_price = pd.read_excel(price_data, skiprows=[0])
df_price['date'] = pd.to_datetime(df_price['Date'])

new_df = pd.merge(new_df, df_price, how='inner', on='date')

In [626]:
new_df.head(10)

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,ratio (t-6),negatives (t-7),positives (t-7),ratio (t-7),Date,High,Low,Upper bound,Lower Bound,Close
0,2019-05-04,21.0,11.0,0.343750,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,5/4/19,5849.500000,5709.740234,7311.875000,4282.305175,5794.140137
1,2019-05-03,43.0,20.0,0.317460,39.0,21.0,0.350000,18.0,11.0,0.379310,...,0.404762,32.0,23.0,0.418182,5/3/19,5901.359863,5596.149902,7376.699829,4197.112426,5840.080078
2,2019-05-02,39.0,21.0,0.350000,18.0,11.0,0.379310,18.0,8.0,0.307692,...,0.418182,29.0,27.0,0.482143,5/2/19,5891.899902,5484.779785,7364.874878,4113.584839,5753.379883
3,2019-05-01,18.0,11.0,0.379310,18.0,8.0,0.307692,15.0,7.0,0.318182,...,0.482143,24.0,23.0,0.489362,5/1/19,5531.149902,5387.290039,6913.937378,4040.467529,5500.720215
4,2019-04-30,18.0,8.0,0.307692,15.0,7.0,0.318182,30.0,9.0,0.230769,...,0.489362,23.0,22.0,0.488889,4/30/19,5417.169922,5346.700195,6771.462403,4010.025146,5389.540039
5,2019-04-29,15.0,7.0,0.318182,30.0,9.0,0.230769,25.0,17.0,0.404762,...,0.488889,25.0,17.0,0.404762,4/29/19,5372.229980,5224.640137,6715.287475,3918.480103,5350.640137
6,2019-04-28,30.0,9.0,0.230769,25.0,17.0,0.404762,32.0,23.0,0.418182,...,0.404762,16.0,10.0,0.384615,4/28/19,5299.509766,5162.810059,6624.387207,3872.107544,5237.959961
7,2019-04-27,25.0,17.0,0.404762,32.0,23.0,0.418182,29.0,27.0,0.482143,...,0.384615,19.0,7.0,0.269231,4/27/19,5313.700195,5218.419922,6642.125244,3913.814941,5272.450195
8,2019-04-26,32.0,23.0,0.418182,29.0,27.0,0.482143,24.0,23.0,0.489362,...,0.269231,9.0,12.0,0.571429,4/26/19,5282.970215,5192.450195,6603.712769,3894.337646,5229.479980
9,2019-04-25,29.0,27.0,0.482143,24.0,23.0,0.489362,23.0,22.0,0.488889,...,0.571429,27.0,11.0,0.289474,4/25/19,5361.939941,5104.180176,6702.424926,3828.135132,5234.089844


In [627]:
new_df['label'] = new_df['Close'].shift() > new_df['Close']
new_df['label'] = new_df['label'].astype(int)

In [628]:
new_df.head(10)

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,negatives (t-7),positives (t-7),ratio (t-7),Date,High,Low,Upper bound,Lower Bound,Close,label
0,2019-05-04,21.0,11.0,0.343750,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,5/4/19,5849.500000,5709.740234,7311.875000,4282.305175,5794.140137,0
1,2019-05-03,43.0,20.0,0.317460,39.0,21.0,0.350000,18.0,11.0,0.379310,...,32.0,23.0,0.418182,5/3/19,5901.359863,5596.149902,7376.699829,4197.112426,5840.080078,0
2,2019-05-02,39.0,21.0,0.350000,18.0,11.0,0.379310,18.0,8.0,0.307692,...,29.0,27.0,0.482143,5/2/19,5891.899902,5484.779785,7364.874878,4113.584839,5753.379883,1
3,2019-05-01,18.0,11.0,0.379310,18.0,8.0,0.307692,15.0,7.0,0.318182,...,24.0,23.0,0.489362,5/1/19,5531.149902,5387.290039,6913.937378,4040.467529,5500.720215,1
4,2019-04-30,18.0,8.0,0.307692,15.0,7.0,0.318182,30.0,9.0,0.230769,...,23.0,22.0,0.488889,4/30/19,5417.169922,5346.700195,6771.462403,4010.025146,5389.540039,1
5,2019-04-29,15.0,7.0,0.318182,30.0,9.0,0.230769,25.0,17.0,0.404762,...,25.0,17.0,0.404762,4/29/19,5372.229980,5224.640137,6715.287475,3918.480103,5350.640137,1
6,2019-04-28,30.0,9.0,0.230769,25.0,17.0,0.404762,32.0,23.0,0.418182,...,16.0,10.0,0.384615,4/28/19,5299.509766,5162.810059,6624.387207,3872.107544,5237.959961,1
7,2019-04-27,25.0,17.0,0.404762,32.0,23.0,0.418182,29.0,27.0,0.482143,...,19.0,7.0,0.269231,4/27/19,5313.700195,5218.419922,6642.125244,3913.814941,5272.450195,0
8,2019-04-26,32.0,23.0,0.418182,29.0,27.0,0.482143,24.0,23.0,0.489362,...,9.0,12.0,0.571429,4/26/19,5282.970215,5192.450195,6603.712769,3894.337646,5229.479980,1
9,2019-04-25,29.0,27.0,0.482143,24.0,23.0,0.489362,23.0,22.0,0.488889,...,27.0,11.0,0.289474,4/25/19,5361.939941,5104.180176,6702.424926,3828.135132,5234.089844,0


In [629]:
new_df = new_df.drop(['High', 'Low', 'Upper bound', 'Lower Bound', 'Close', 'Date'], axis=1)

In [630]:
new_df.head(10)

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,negatives (t-5),positives (t-5),ratio (t-5),negatives (t-6),positives (t-6),ratio (t-6),negatives (t-7),positives (t-7),ratio (t-7),label
0,2019-05-04,21.0,11.0,0.343750,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0
1,2019-05-03,43.0,20.0,0.317460,39.0,21.0,0.350000,18.0,11.0,0.379310,...,30.0,9.0,0.230769,25.0,17.0,0.404762,32.0,23.0,0.418182,0
2,2019-05-02,39.0,21.0,0.350000,18.0,11.0,0.379310,18.0,8.0,0.307692,...,25.0,17.0,0.404762,32.0,23.0,0.418182,29.0,27.0,0.482143,1
3,2019-05-01,18.0,11.0,0.379310,18.0,8.0,0.307692,15.0,7.0,0.318182,...,32.0,23.0,0.418182,29.0,27.0,0.482143,24.0,23.0,0.489362,1
4,2019-04-30,18.0,8.0,0.307692,15.0,7.0,0.318182,30.0,9.0,0.230769,...,29.0,27.0,0.482143,24.0,23.0,0.489362,23.0,22.0,0.488889,1
5,2019-04-29,15.0,7.0,0.318182,30.0,9.0,0.230769,25.0,17.0,0.404762,...,24.0,23.0,0.489362,23.0,22.0,0.488889,25.0,17.0,0.404762,1
6,2019-04-28,30.0,9.0,0.230769,25.0,17.0,0.404762,32.0,23.0,0.418182,...,23.0,22.0,0.488889,25.0,17.0,0.404762,16.0,10.0,0.384615,1
7,2019-04-27,25.0,17.0,0.404762,32.0,23.0,0.418182,29.0,27.0,0.482143,...,25.0,17.0,0.404762,16.0,10.0,0.384615,19.0,7.0,0.269231,0
8,2019-04-26,32.0,23.0,0.418182,29.0,27.0,0.482143,24.0,23.0,0.489362,...,16.0,10.0,0.384615,19.0,7.0,0.269231,9.0,12.0,0.571429,1
9,2019-04-25,29.0,27.0,0.482143,24.0,23.0,0.489362,23.0,22.0,0.488889,...,19.0,7.0,0.269231,9.0,12.0,0.571429,27.0,11.0,0.289474,0


In [631]:
new_df.to_csv(output_dataset_name)